In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics
from flask import Flask, request, render_template,jsonify
from flask_cors import CORS
import re
import math
from utility.data_cleaning import *
from utility.data_preprocessing import *
from utility.model_building import *
from config.configuration import *


app = Flask(__name__)


#import model and encoder  
with open('./data/training/output/lgb_model_20231206.pkl', 'rb') as file:
    model=pickle.load(file)
with open('./data/training/output/encoder_20231206.pkl', 'rb') as file:
    label_encoders=pickle.load(file)
modes_dict_path = './data/training/output/mode_values_dict_20231206.pkl'
with open(modes_dict_path, 'rb') as file:
    modes_dict = pickle.load(file)
print(modes_dict)
    
@app.route("/predict_api", methods=['POST'])
def RiskPrediction():
    json_ = request.json
    df= pd.DataFrame([json_])
    display(df)
    df.replace('',np.nan,inplace=True)
    for col in df.columns: 
        if col in modes_dict:
            df[col]=df[col].fillna(modes_dict[col])
            

    display(df)
    df_clean = clean_data(df)
    display(df_clean)
    for col, encoder in label_encoders.items():
        print(f"Column: {col}")
        print(f"Classes: {encoder.classes_}")
        print("------")
    for col in label_encoders:
        df_clean[col] = label_encoders[col].transform(df_clean[col])
    
    predicted_loss_amount= model.predict(df_clean)
    output = predicted_loss_amount[0]
    
    
    
   
#     columns_to_fillna = [col for col in columns_to_string if col in modes_dict]
#     df[columns_to_fillna] = df[columns_to_fillna].apply(lambda col: col.fillna(modes_dict[col.name]))
#     df_clean = clean_data(df)
#     columns_to_fillna = [col for col in columns_to_string if col in df.columns]
#     for col in label_encoders:
#         # Fit on the union of training and test data
#         combined_data = pd.concat([pd.DataFrame(label_encoders[col].classes_.ravel()), pd.DataFrame(df_clean[col].ravel())])
#     label_encoders[col].fit(combined_data)
#     df_clean[col] = label_encoders[col].transform(df_clean[col])
#     predicted_loss_amount = model.predict(df_clean)
#     output = predicted_loss_amount[0]
#     #df = df.apply(lambda col: col.fillna(modes_dict[col.name]) if col.name in columns_to_string else col)
#     print(modes_dict)
#     display(df)
#     df_clean = clean_data(df)
#     display(df)
#     for col in label_encoders:
#         df_clean[col] = label_encoders[col].transform(df_clean[col])
#     predicted_loss_amount= model.predict(df_clean)
#     output = predicted_loss_amount[0]

    print(output)
    
    return jsonify({"prediction":output})
if __name__ == '__main__':
    #app.run()
    app.run(host='0.0.0.0', port=80)

{'Jurisdiction': 'Pennsylvania', 'CoverageType': 'BOP-Building', 'Deductible': 'FixedDollar - 2500.00', 'Limit': 'Blanket Group 1 Blanket - 48075046.00', 'InjuryCause': 'WATER', 'Comments': 'ClassCode:65146', 'ConstructionTypeDesc': 'Frame', 'ZipCode': '18102'}
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:80
 * Running on http://192.168.152.90:80
Press CTRL+C to quit


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Maryland,BOP-Building,,Standard - 7096952.00,WATER,2008,4,ClassCode:65147,,17319,8308610.34,35000,7096952


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Maryland,BOP-Building,FixedDollar - 2500.00,Standard - 7096952.00,WATER,2008,4,ClassCode:65147,Frame,17319,8308610.34,35000,7096952


,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,maryland,bop-building,fixeddollar - 2500.00,standard - 7096952.00,water,2008,4,classcode:65147,frame,17319,8308610.34,35000,7096952


,Jurisdiction,CoverageType,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit,limit_type,Limit_val,Deductible_type,Deductible_val,YearGap
0,maryland,bop-building,water,4,classcode:65147,frame,17319,8308610.34,35000,7096952,standard,7096952.0,fixeddollar,2500,15


,Jurisdiction,CoverageType,InjuryCause,Stories,ClassCode,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit,limit_type,Limit_val,Deductible_type,Deductible_val,YearGap
0,maryland,bop-building,water,4,classcode:65147,frame,17319,8308610.34,35000,7096952,standard,7096952.0,fixeddollar,2500,15


127.0.0.1 - - [06/Dec/2023 09:20:28] "POST /predict_api HTTP/1.1" 200 -


Column: Jurisdiction
Classes: ['delaware' 'dist. of columbia' 'maryland' 'north carolina' 'ohio'
 'pennsylvania' 'virginia']
------
Column: CoverageType
Classes: ['bop-building']
------
Column: InjuryCause
Classes: ['electric current' 'fire' 'freezing' 'hail' 'other perils'
 'vandalism & malicious mischief' 'vehicles' 'water' 'wind']
------
Column: ClassCode
Classes: ['classcode:09061' 'classcode:09651' 'classcode:0966109661'
 'classcode:59997' 'classcode:59999' 'classcode:63611' 'classcode:63981'
 'classcode:65132' 'classcode:65141' 'classcode:65142' 'classcode:65144'
 'classcode:65145' 'classcode:6514565145' 'classcode:65146'
 'classcode:65147' 'classcode:65171' 'classcode:69145' 'classcode:75521']
------
Column: ConstructionTypeDesc
Classes: ['frame' 'joistedmasonry' 'masonrynoncombustible']
------
Column: ZipCode
Classes: ['15045' '15108' '15120' '15133' '15144' '15145' '15206' '15210' '15212'
 '15217' '15218' '15232' '15236' '15301' '15456' '15601' '15620' '15644'
 '15661' '15714'

In [1]:
import pandas as pd
data= {
    "Jurisdiction":"Delaware",
    "CoverageType":"BOP-Building",
    "Deductible":"FixedDollar - 1000.00",
    "Limit":"Standard - 217000.00",
    "InjuryCause":"WIND",
    "YearBuilt":"1958",
    "Stories":"1",
    "Comments":"ClassCode:65141",
    "ConstructionTypeDesc":"Frame",
    "ZipCode":"19973",
    "E2Value":"217236",
    "SquareFootage":"1480",
    "BuildingLimit":"217000"
}
df=pd.DataFrame([data])
display(df)

,Jurisdiction,CoverageType,Deductible,Limit,InjuryCause,YearBuilt,Stories,Comments,ConstructionTypeDesc,ZipCode,E2Value,SquareFootage,BuildingLimit
0,Delaware,BOP-Building,FixedDollar - 1000.00,Standard - 217000.00,WIND,1958,1,ClassCode:65141,Frame,19973,217236,1480,217000
